In [1]:
from pathlib import Path

SED_HOME = Path.home().joinpath('Documents').joinpath('sed')
CDPP_DIR = SED_HOME.joinpath('CDPP')
CDPP_EXAMPLES = CDPP_DIR.joinpath('examples')
CDPP_SRC = CDPP_DIR.joinpath('src')
CDPP_SCRIPTS = CDPP_DIR.joinpath('scripts')
BASE_BIN = CDPP_SRC.joinpath('bin')
CDPP_ATOMICS = CDPP_SRC.joinpath('cd++/atomics')

In [2]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/dc-sed-tp1-2022/model/

cat manager.ma

[top]
components : manager@manager
in : in_manager
out : out1_manager out2_manager out3_manager out4_manager out5_manager out6_manager out7_manager out8_manager out9_manager out10_manager done_manager
link : out1@manager out1_manager
link : out2@manager out2_manager
link : out3@manager out3_manager
link : out4@manager out4_manager
link : out5@manager out5_manager
link : out6@manager out6_manager
link : out7@manager out7_manager
link : out8@manager out8_manager
link : out9@manager out9_manager
link : out10@manager out10_manager
link : in_manager in@manager


In [3]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/dc-sed-tp1-2022/model/
head manager.ev

00:00:00:15 in_manager 1
00:00:00:30 in_manager 5
00:00:00:45 in_manager 10
00:00:01:20 in_manager 15
00:00:01:40 in_manager 20
00:00:03:10 in_manager 24
00:00:04:45 in_manager 25
00:03:08:45 in_manager 26

In [4]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/dc-sed-tp1-2022/src/

bin/cd++ -m ../model/manager.ma  \
         -e ../model/manager.ev  \
         -l ../manager.log \
         -o ../manager.out \
         -t 01:00:00:00

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from ../model/manager.ma
Loading events from ../model/manager.ev
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: ../manager.log
Output to: ../manager.out
Tolerance set to: 1e-08
Configuration to show real nu

# Resultados

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# definimos los path a los archivos de interés
OUT_FILENAME = str(CDPP_EXAMPLES)+"/dc-sed-tp1-2022/manager.out"

# definimos los nombres de las columnas en los dataframes de pandas
TIME_COL = 'time'
PORT_COL = 'port'
VALUE_COL = 'value'

# conversion a floar
def parse_value(value: str):
    is_list = value.strip().startswith("[") and value.strip().endswith("]")
    if is_list:
        return tuple(float(num) for num in value.replace('[', '').replace(']', '').split(', '))
    return float(value)

# conversion VTime a float
def time_to_secs(time):
    h, m, s, ms, r = time.split(':')
    return float(h)*60*60. + float(m)*60. + float(s) + float(ms)/1000. + float(r)/1000.

In [6]:
df_converters = {
    VALUE_COL: parse_value,
    TIME_COL: time_to_secs
}

df_converters = {
    VALUE_COL: parse_value,
    TIME_COL: time_to_secs
}

salida = pd.read_csv(OUT_FILENAME,
                     delimiter=r'(?<!,)\s+',
                     engine='python',  # C engine doesnt work for regex
                     # Se puede usar esto para convertir los valores directamente:
                     converters=df_converters,
                     names=[TIME_COL, PORT_COL, VALUE_COL]
                    )
salida

,time,port,value
0,0.031,out1_manager,0.0
1,0.031,out2_manager,0.0
2,0.031,out3_manager,0.0
3,0.031,out4_manager,0.0
4,0.031,out5_manager,0.0
...,...,...,...
65,188.046,out6_manager,0.0
66,188.046,out7_manager,0.0
67,188.046,out8_manager,0.0
68,188.046,out9_manager,0.0
